In [40]:
data = """1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,20~1,1,21
"""
#data = open('puzzle.data').read()

from dataclasses import dataclass, replace
from functools import reduce

@dataclass
class Brick:
    x1: int
    y1: int
    z1: int
    x2: int
    y2: int
    z2: int

    def __post_init__(self):
        self.x1, self.x2 = sorted((self.x1, self.x2))
        self.y1, self.y2 = sorted((self.y1, self.y2))
        self.z1, self.z2 = sorted((self.z1, self.z2))
        if self.x1 != self.x2:
            self.cubes = set((x, self.y1, self.z1) for x in range(self.x1, self.x2+1))
        elif self.y1 != self.y2:
            self.cubes = set((self.x1, y, self.z1) for y in range(self.y1, self.y2+1))
        else:
            self.cubes = set((self.x1, self.y1, z) for z in range(self.z1, self.z2+1))

    @property
    def at_bottom(self):
        return 0 in (self.z1, self.z2)

    def fall(self):
        self.z1 -= 1
        self.z2 -= 1
        self.cubes = set((x, y, z - 1) for x, y, z in self.cubes)
    
bricks = []
for line in data.splitlines():
    x1, y1, z1, x2, y2, z2 = map(int, line.replace('~', ',').split(','))
    bricks.append(Brick(x1, y1, z1, x2, y2, z2))

# let them fall
all_cubes = reduce(lambda a, b: a | b.cubes, bricks, set())

def iterate(bricks, all_cubes):
    bricks_falling = 0
    for brick in bricks:
        if brick.at_bottom:
            continue
        falling = False
        while True:
            brick_one_lower = replace(brick, z1=brick.z1 - 1, z2= brick.z2 - 1)
            test_cubes = all_cubes - brick.cubes
            if brick_one_lower.cubes & test_cubes or brick.at_bottom:
                break
            brick.fall()
            all_cubes = test_cubes | brick.cubes
            falling = True
        if falling:
            bricks_falling += 1
    return bricks, all_cubes, bricks_falling

# let them fall
while True:
    bricks, all_cubes, bricks_falling = iterate(bricks, all_cubes)
    if not bricks_falling:
        break



In [41]:
# test disintegration
bricks_falling = list()

for brick in bricks:
    test_cubes = all_cubes - brick.cubes
    test_bricks = [replace(b) for b in bricks if b != brick]
    while True:
        test_bricks, test_cubes, b = iterate(test_bricks, test_cubes)
        if not b:
            break
    bricks_falling.append(len([b for b in test_bricks if b not in bricks]))

print(len([b for b in bricks_falling if b == 0]))
print(sum(bricks_falling))

5
7
